In [1]:
import pyreadr, pickle, csv, re
import numpy as np
import pandas as pd


import torch
import torch.nn as nn

from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle

In [2]:
N = 300 # How many genes to extract

path = '/home/sam/scRNAseq/Xenium/Retina_expMatrix_clean_final2.RData'
rdata = pyreadr.read_r(path) # WS
# rdata = pyreadr.read_r('/home/sam/Downloads/keyGeneExpressionBySubtype.RData') #Laptop
# genes = rdata['key_genes']

In [3]:
# Load data
df = rdata['Retina_expMatrix_candidateGenes']
df['Cluster'] = df['Cluster'].apply(lambda x: x if len(x.split('_')[0]) == 2 else '0' + x) # Standardize cluster names

def encode_class(arr):
    '''This function will encode subtypes' cell classesbased on expert rules and is not intended for decoding'''
    custom_array = []

    for value in arr:
        if re.match(r'^\d{2}_', value):
            custom_array.append(0)
        elif value.startswith('AC_'):
            custom_array.append(1)
        elif value.endswith('Photoreceptors'):
            custom_array.append(2)
        elif value == '0MG (Mueller Glia)':
            custom_array.append(3)
        elif value.startswith('0BC'):
            custom_array.append(4)
        elif value.startswith('0RBC'):
            # Note this duplication is for simplicity of handling the 2 BC naming conventions
            custom_array.append(4)
        else:
            custom_array.append(5)
    return custom_array

class_arr = encode_class(df['Cluster'])

# Encode the categoric response 
le = LabelEncoder()
df['Cluster'] = le.fit_transform(df['Cluster'])

# Move the response to the end for simply manipulation
cluster_col = df.pop('Cluster')
dataset_col = df.pop('Dataset')
df.insert(len(df.columns), 'Cluster', cluster_col)
df.insert(len(df.columns), 'Class', class_arr)

display(df)

,Kcnip4,Isl2,Glra1,Zic1,Syndig1l,Isl1,Pou3f1,Mmp9,Grm5,Cpne4,...,Kcnab2,Glrb,Rbpms,Vamp1,Cspg4,Kcnq1ot1,Cdh5,Foxp1,Cluster,Class
rownames,,,,,,,,,,,,,,,,,,,,,
Bipolar1_CCCACAAGACTA,0.367305,0.000000,0.464855,0.0,0.000000,0.464855,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,18,4
Bipolar1_CAAAGCATTTGC,0.323303,0.000000,0.323303,0.0,0.000000,1.785388,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,19,4
Bipolar1_CTTTTGATTGAC,0.000000,0.000000,0.000000,0.0,0.000000,0.732856,0.0,0.000000,0.307734,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,20,4
Bipolar1_GCTCCAATGACA,0.000000,0.000000,0.000000,0.0,0.000000,1.701798,0.0,0.000000,0.296325,0.000000,...,0.296325,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,24,4
Bipolar1_AAATACCCTCAT,0.000000,0.000000,0.000000,0.0,0.000000,0.786085,0.0,0.000000,0.585854,0.000000,...,0.000000,0.335230,0.000000,0.000000,0.0,0.000000,0.0,0.000000,15,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
aRGC10_TTTGTCAAGGAACTGC.1,2.297165,0.639005,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.639005,1.699822,...,1.025751,0.000000,1.699822,0.639005,0.0,1.303941,0.0,0.000000,1,0
aRGC10_TTTGTCAAGTAGGTGC.1,2.338764,0.616090,0.000000,0.0,0.000000,0.616090,0.0,0.994491,0.000000,1.940426,...,0.994491,0.000000,1.659821,0.616090,0.0,0.000000,0.0,0.616090,1,0
aRGC10_TTTGTCACATGTAAGA.1,0.902590,0.000000,0.000000,0.0,0.549849,1.369139,0.0,0.000000,0.000000,0.549849,...,0.549849,0.902590,1.813348,0.902590,0.0,0.000000,0.0,0.000000,7,0


In [4]:
def create_mapping(df):
    # Extract the unique pairs of encoded cluster values and their corresponding class encodings
    unique_pairs = df[['Cluster', 'Class']].drop_duplicates()
    
    # Create a dictionary mapping from Cluster to Class
    mapping = dict(zip(unique_pairs['Cluster'], unique_pairs['Class']))
    
    return mapping

# Usage
mapping = create_mapping(df)


In [5]:
# Shuffle the data
df = shuffle(df, random_state=42)

# Split the data into input features and labels
X = df.iloc[:, :-2].values.astype(np.float32)
X = np.round(X*100)/100
y = df.iloc[:, -2].values.astype(np.compat.long)

# Convert data to PyTorch tensors
X = torch.from_numpy(X)
y = torch.from_numpy(y)

# Split the data into training and test sets
train_size = int(0.8 * len(df))
train_X, test_X = X[:train_size], X[train_size:]
train_y, test_y = y[:train_size], y[train_size:]

In [6]:
class Net(nn.Module):
    def __init__(self, input_size, num_classes,
                 h1_size, h2_size, h3_size, dropout_prob):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, h1_size)
        # self.fc2 = nn.Linear(h1_size, h2_size)
        # self.fc3 = nn.Linear(h2_size, h3_size)
        self.fc4 = nn.Linear(h1_size, num_classes)
        self.dropout = nn.Dropout(p=dropout_prob)

    def forward(self, x):
        x = nn.functional.relu(self.fc1(x))
        # x = nn.functional.relu(self.fc2(x))
        # x = self.dropout(x)
        # x = nn.functional.relu(self.fc3(x))
        # x = self.dropout(x)
        x = self.fc4(x)
        return nn.functional.log_softmax(x, dim=1)
    


In [10]:
# This cell is an experiment in training currciulumn
# The idea is to specifically train networks to learn one class only
# To do this we will train it head on against one other class ata a time
# When trainign is complete the network will have been exposed to the desired class at least 44 times
# It will see all non-desired classes at least once
# In this instantiation the network will not be weighted ahead of time to know what class is the desired class
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split

# Step 1: Prepare training data
def head2head(train_y, train_X, desired, noise):
    '''This function will take a desired category and a noise category and create a paired down dataset
    The paired down data set will be an equal mix of desired and noise
    If there are not enough noise examples, a random set will be duplicated to make the lengths match
    If there are too many noise samples, a random set will be excluded'''

    # Isolate the desired instances
    TS_desired_y = train_y[train_y == desired]
    TS_desired_X = train_X[train_y == desired]
    TS_noise_y = train_y[train_y == noise]
    TS_noise_X = train_X[train_y == noise]
    # Check if the desired class is longer, if so balance it with replicates from te noise
    if len(TS_desired_y) > len(TS_noise_y):
        delta = len(TS_desired_y) - len(TS_noise_y)
        indice = np.random.choice( range(len(TS_noise_y)), delta)
        indice = torch.tensor(indice)
        extra_y = TS_noise_y[indice]
        extra_X = TS_noise_X[indice]
        TS_noise_y = torch.cat((TS_noise_y,extra_y))
        TS_noise_X = torch.cat((TS_noise_X,extra_X))
    # Check if the desired class is shorter, if so randomely discard examples from the other
    elif len(TS_desired_y) < len(TS_noise_y):
        indice = np.random.choice( range(len(TS_noise_y)), len(TS_desired_y))
        indice = torch.tensor(indice)
        TS_noise_y = TS_noise_y[indice]
        TS_noise_X = TS_noise_X[indice]
    # Combine the two balanced data sets
    TS_y = torch.cat((TS_noise_y,TS_desired_y))
    TS_X = torch.cat((TS_noise_X,TS_desired_X))
    # Shuffle the data
    indice = np.random.choice( range(len(TS_y)), len(TS_y))
    indice = torch.tensor(indice)
    TS_y = TS_y[indice]
    TS_X = TS_X[indice]
    return TS_y, TS_X


def DesiredGeneCurriculum(train_y, train_X, desired=1):
    if desired!=None:
        # Create a training set using this function to create my desired class biased training set
        noise_inds = [i for i in np.unique(train_y) if i != desired]

    # Move the data to the GPU
    # Set the device to use (GPU if available, otherwise CPU)
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    # Convert the input data to the appropriate data type for the GPU
    train_X = train_X.to(device)
    train_y = train_y.to(device)

    ts_datasets = []
    if desired!=None:
        for noise_ind in noise_inds:
            TS_y, TS_X = head2head(train_y, train_X, desired=desired, noise=noise_ind)
            ts_datasets.append(torch.utils.data.TensorDataset(TS_X, TS_y))
    else:
        ts_datasets.append(torch.utils.data.TensorDataset(train_X, train_y))

    # Concatenate all datasets into one training set
    training_set = torch.utils.data.ConcatDataset(ts_datasets)
    
    return training_set


class CustomLoss(nn.Module):
    def __init__(self, mapping, target_subtype, class_weight):
        super(CustomLoss, self).__init__()
        self.target_subtype = target_subtype
        self.cross_entropy_loss = nn.CrossEntropyLoss()
        self.class_weight = class_weight
        self.subtype_weight = 1-class_weight
        self.encoding_map = mapping
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    
    def probability_mapping(self, probabilities):
        # Create an empty tensor for intermediate probabilities
        intermediate_prob = torch.zeros(probabilities.shape[0], len(set(self.encoding_map.values())), device=self.device)

        # Create a tensor for the mapping indices
        mapping_indices = torch.tensor(list(self.encoding_map.values()), device=self.device)

        # Vectorized operation to sum probabilities for each intermediate class
        for i in range(len(set(self.encoding_map.values()))):
            intermediate_prob[:, i] = probabilities[:, mapping_indices == i].sum(dim=1)

        return intermediate_prob

    def forward(self, outputs, targets, delta = 0):
        # This delta catch ensures simple math errors during training from resulting in unreasonable delta values
        if delta < 0:
            delta = 0
        elif delta > 1:
            delta = 1
        if self.target_subtype == None:
            filtered_outputs = outputs
            filtered_targets = targets
        else:
            # Filter outputs and targets to keep only the values corresponding to the target_subtype
            filtered_outputs = torch.zeros_like(outputs)
            filtered_outputs[:, self.target_subtype] = outputs[:, self.target_subtype]

            filtered_targets = torch.where(targets == self.target_subtype, targets, torch.tensor(0, dtype=targets.dtype))

        # Convert filtered targets to class encodings using the mapping
        y_true_classes = torch.tensor([self.encoding_map[int(t)] for t in filtered_targets], device=self.device)

        # Convert logits to probabilities
        probabilities = torch.exp(filtered_outputs)  # Since log_softmax was used in the model

        # Map to intermediate probabilities
        y_pred_classes = self.probability_mapping(probabilities)
        
        # Convert probabilities to log probabilities for NLLLoss
        log_probabilities = torch.log(y_pred_classes + 1e-6)  # Adding a small value for numerical stability

        # Calculate the CrossEntropyLoss using the filtered outputs and targets for subtypes
        loss_sc = self.cross_entropy_loss(filtered_outputs, filtered_targets)

        # Calculate NLLLoss using log probabilities and true classes
        nll_loss = nn.NLLLoss()
        loss_c = nll_loss(log_probabilities, y_true_classes)
        
        # Weight the losses based on current stage of training or experiment
        loss = loss_sc*(self.subtype_weight+delta) + loss_c*(self.class_weight-delta)

        return loss


def QuickNN(training_set, n, num_epochs, batch_size, mapping,
            target_subtype=None, class_weight=1, deltaRate = 0,
            l1_lambda = 0.0, stopEarly = 10, visualize=False):
    # Set the device to use (GPU if available, otherwise CPU)
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    # Step 2: Define neural network
    # Instantiate the neural network model
    inputs = training_set[0][0].shape[0]
    model = Net(input_size=inputs, num_classes=n,
                h1_size=n*2, h2_size=n*3, h3_size=n*4, dropout_prob=0
                ).to(device)

    # Define the loss function and optimizer
    criterion = CustomLoss(mapping, target_subtype, class_weight)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=l1_lambda, amsgrad=True)

    # Define the learning rate scheduler
    lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', 
                                                          factor=0.05, patience=5)
    
    # Step 3: Set up training loop
    train_loader = DataLoader(training_set, batch_size=batch_size, shuffle=True)
    best_loss = float('inf')  # initialize the best validation loss
    early_stop_counter = 0  # initialize the early stopping counter

    if stopEarly > 0:
        print("Early Stopping Initialized")
        # Create the validation set
        val_size = int(len(training_set) * 0.2) # Use 20% of the training set for validation
        val_set, train_set = random_split(training_set, [val_size, len(training_set) - val_size])
        val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=True)

    delta = 0
    for epoch in range(num_epochs):
        epoch_loss = 0.0
        for batch_idx, (inputs, targets) in enumerate(train_loader):
            if len(inputs) == 0:
                continue
            inputs, targets = inputs.to(device), targets.to(device)

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, targets, delta = delta)
            epoch_loss += loss.item()

            # Backward Pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
        # Increment delta at the end of each epoch
        if deltaRate > 0:
            delta += deltaRate
            delta = min(delta, 1)  # Ensure delta does not exceed 1
            print(f'\rEpoch {epoch+1}/{num_epochs}, Current Delta: {delta:.3f}', end='')

        if stopEarly > 0:
            # Define the learning rate scheduler
            lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', 
                                                                factor=0.05, patience=5)
            # Evaluate the model on the validation set
            with torch.no_grad():
                val_loss = 0.0
                for inputs, targets in val_loader:
                    inputs, targets = inputs.to(device), targets.to(device)
                    outputs = model(inputs)
                    val_loss += criterion(outputs, targets).item()
            val_loss /= len(val_loader)

            # Check if the validation loss has improved
            if np.round(val_loss,5) < best_loss:
                best_loss = val_loss
                early_stop_counter = 0
            else:
                early_stop_counter += 1
                if early_stop_counter >= stopEarly:  # if the validation loss hasn't improved for 10 epochs, stop training
                    print(f"Early stopping at: Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}, Learning rate: {optimizer.param_groups[0]['lr']:.6f}")
                    break
        else:
            # Update the learning rate using the scheduler
            lr_scheduler.step(loss)


        if visualize:
            # Print the training loss and learning rate after every epoch
            print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}, Learning rate: {optimizer.param_groups[0]['lr']:.6f}")
        
    return model

def TestModel(test_X, test_y, model, le, visualize=True):
    # Set the device to use (GPU if available, otherwise CPU)
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    test_X = test_X.to(device)
    test_y = test_y.to(device)

    # Evaluate the model on the test set
    with torch.no_grad():
        outputs = model(test_X)
        _, predicted = torch.max(outputs.data, 1)

    results = pd.DataFrame()
    for i in range(min(y), max(y)+1):
        cells = sum(test_y==i).item()
        test_y_i = test_y==i
        y_pred_i = predicted==i
        TP = sum((test_y_i==1) & (y_pred_i==1)).item()
        FP = sum((test_y_i==0) & (y_pred_i==1)).item()
        TN = sum((test_y_i==0) & (y_pred_i==0)).item()
        FN = sum((test_y_i==1) & (y_pred_i==0)).item()
        TPR = TP / np.where(TP+FN == 0, np.nan, TP+FN)
        TNR = TN / np.where(TN+FP == 0, np.nan, TN+FP)
        Prec = TP / np.where(TP+FP == 0, np.nan, TP+FP)
        F1 = 2 * (Prec * TPR) / np.where((Prec + TPR) == 0, np.nan, Prec + TPR)
        Accuracy = (TP+TN) / np.where(TP+FP+FN+TN == 0, np.nan, TP+FP+FN+TN)

        res_i = {'Cluster' : le.inverse_transform([i])[0],
            'cells' : cells,
            'TP' : TP,
            'FP' : FP,
            'TN' : TN,
            'FN' : FN,
            'TPR' : TPR,
            'TNR' : TNR,
            'Prec' : Prec,
            'F1' : F1,
            'Accuracy' : Accuracy}
        
        res_i = pd.DataFrame([res_i])
        results = pd.concat([results,res_i], ignore_index=True)
        
    if visualize:
        display(results.sort_values(by=['Cluster']))

    return results

In [8]:
n = len(np.unique(y))
num_epochs = 100 # specify the number of epochs to train for
batch_size = 32 # specify the batch size for training

# Convert train_X and test_X to PyTorch tensors on the GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
train_X = train_X.to(device)
test_X = test_X.to(device)

training_set = DesiredGeneCurriculum(train_y, train_X, desired=None)
curriculum_dict = {}
for seed in range(0,18,3):
    for w in (0, 0.001, 0.005, 0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 1.0):
        torch.manual_seed(seed)
        np.random.seed(seed)
        
        model = QuickNN(training_set, n, num_epochs, batch_size, mapping, 
                        class_weight=w,
                        stopEarly=0, visualize=False)
        results = TestModel(test_X, test_y, model, le, visualize=False)
        curriculum_dict[f'Experiment1 classweight {w} replicate {seed}'] = {}
        curriculum_dict[f'Experiment1 classweight {w} replicate {seed}']['model'] = model

        curriculum_dict[f'Experiment1 classweight {w} replicate {seed}']['results'] = results

        # Open a file for writing
        with open('/home/sam/scRNAseq/Xenium/ClassVsSubclass/Experiment1/Experiment1.pkl', 'wb') as f:
            # Use pickle to dump the list to the file
            pickle.dump(curriculum_dict, f)
        results.to_csv(f'/home/sam/scRNAseq/Xenium/ClassVsSubclass/Experiment1/Experiment1_classVSsubclass_classweight_{w}_result_replicate{seed}.csv', index=False)
        print(f'Experiment1 classweight {w} replicate {seed} completed and saved, mean F1={np.mean(results["F1"])}')

Experiment1 classweight 0 replicate 0 completed and saved, mean F1=0.9146507206905338
Experiment1 classweight 0.001 replicate 0 completed and saved, mean F1=0.914254304828893
Experiment1 classweight 0.005 replicate 0 completed and saved, mean F1=0.9100811853350514
Experiment1 classweight 0.01 replicate 0 completed and saved, mean F1=0.9074297767925896
Experiment1 classweight 0.05 replicate 0 completed and saved, mean F1=0.8819825471415782
Experiment1 classweight 0.1 replicate 0 completed and saved, mean F1=0.8701211697922181
Experiment1 classweight 0.25 replicate 0 completed and saved, mean F1=0.8427956831165005
Experiment1 classweight 0.5 replicate 0 completed and saved, mean F1=0.5901410290000679
Experiment1 classweight 0.75 replicate 0 completed and saved, mean F1=0.22979397278903857
Experiment1 classweight 1.0 replicate 0 completed and saved, mean F1=nan
Experiment1 classweight 0 replicate 3 completed and saved, mean F1=0.9068367233033442
Experiment1 classweight 0.001 replicate 3 c

In [13]:
n = len(np.unique(y))
num_epochs = 100 # specify the number of epochs to train for
batch_size = 32 # specify the batch size for training

# Convert train_X and test_X to PyTorch tensors on the GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
train_X = train_X.to(device)
test_X = test_X.to(device)

training_set = DesiredGeneCurriculum(train_y, train_X, desired=None)
curriculum_dict = {}
for seed in range(0,18,3):
    for dr in (0.001, 0.005, 0.01, 0.025, 0.05, 0.1, 0.25, 0.5, 0.75):
        torch.manual_seed(seed)
        np.random.seed(seed)
        
        model = QuickNN(training_set, n, num_epochs, batch_size, mapping, 
                        class_weight=1,deltaRate=dr,
                        stopEarly=0, visualize=False)
        results = TestModel(test_X, test_y, model, le, visualize=False)
        curriculum_dict[f'Experiment2 delta Rate {dr} replicate {seed}'] = {}
        curriculum_dict[f'Experiment2 delta Rate {dr} replicate {seed}']['model'] = model

        curriculum_dict[f'Experiment2 delta Rate {dr} replicate {seed}']['results'] = results

        # Open a file for writing
        with open('/home/sam/scRNAseq/Xenium/ClassVsSubclass/Experiment2/Experiment2.pkl', 'wb') as f:
            # Use pickle to dump the list to the file
            pickle.dump(curriculum_dict, f)
        results.to_csv(f'/home/sam/scRNAseq/Xenium/ClassVsSubclass/Experiment2/Experiment2_classVSsubclass_deltaRate_{dr}_result_replicate{seed}.csv', index=False)
        print(f'\nExperiment2 delta Rate {dr} replicate {seed} completed and saved, mean F1={np.mean(results["F1"])}')

Epoch 100/100, Current Delta: 0.100Experiment2 delta Rate 0.001 replicate 0 completed and saved, mean F1=0.5823608599423999
Epoch 100/100, Current Delta: 0.500Experiment2 delta Rate 0.005 replicate 0 completed and saved, mean F1=0.4141576553656612
Epoch 100/100, Current Delta: 1.000Experiment2 delta Rate 0.01 replicate 0 completed and saved, mean F1=0.218271578000047
Epoch 100/100, Current Delta: 1.000Experiment2 delta Rate 0.025 replicate 0 completed and saved, mean F1=0.6170545579317271
Epoch 100/100, Current Delta: 1.000Experiment2 delta Rate 0.05 replicate 0 completed and saved, mean F1=0.8934584763750454
Epoch 100/100, Current Delta: 1.000Experiment2 delta Rate 0.1 replicate 0 completed and saved, mean F1=0.9128994743379873
Epoch 100/100, Current Delta: 1.000Experiment2 delta Rate 0.25 replicate 0 completed and saved, mean F1=0.9129833312207418
Epoch 100/100, Current Delta: 1.000Experiment2 delta Rate 0.5 replicate 0 completed and saved, mean F1=0.9151871882300611
Epoch 100/100, C

In [83]:
def compute_feature_importance(model, input_data, target_category):
    input_data.requires_grad = True # tell PyTorch to compute gradients with respect to the input
    model.zero_grad()
    output = model(input_data)
    # compute the negative log likelihood loss between the output and the target category
    loss = nn.functional.nll_loss(output, target_category) 
    # compute the gradients of the loss with respect to the input.
    loss.backward()
    # feature importance as the mean absolute value of the gradients over the batch dimension (i.e., over all input examples).
    feature_importance = input_data.grad.abs().mean(dim=0)
    return feature_importance.to('cpu')

N = 300

# Convert train_X and test_X to PyTorch tensors on the GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
train_X = train_X.to(device)
test_X = test_X.to(device)

# Convert input data to a PyTorch tensor and move it to the GPU
input_data = torch.Tensor(test_X).to(device)

# Compute feature importance for each target category
mdl = curriculum_dict[f'Model all final genes']['model'].to(device)
for i in np.unique(y):
    curriculum_dict[f'Model all final genes'][i] = {}
    target_category = torch.full((input_data.shape[0],), i, device=device, dtype=torch.long)
    curriculum_dict[f'Model all final genes'][i]['feature importance'] = compute_feature_importance(mdl, input_data, target_category)
    top_n_values, curriculum_dict[f'Model all final genes'][i][f'top {N} features'] = torch.topk(curriculum_dict[f'Model all final genes'][i]['feature importance'], N, largest=True)
    curriculum_dict[f'Model all final genes'][i][f'top {N} genes'] = list(df.columns[curriculum_dict[f'Model all final genes'][i][f'top {N} features']])


In [84]:
ordered_gene_list = []
for i in np.unique(y):
    ordered_gene_list.append([le.inverse_transform([i]), curriculum_dict[f'Model all final genes'][i]['top 300 genes']])

In [85]:
# Pandas version
data_dict = {}
for i in np.unique(y):
    label = le.inverse_transform([i])[0]
    values = curriculum_dict[f'Model all final genes'][i]['top 300 genes']
    data_dict[label] = values

# Convert the dictionary to a pandas DataFrame
df = pd.DataFrame.from_dict(data_dict, orient='index')
df.index.name = 'Label'

In [86]:
# Step 1: Create masks for "dd_*" and "AC_*" formats
dd_mask = df.index.str.match(r'^\d{2}_.*')
ac_mask = df.index.str.match(r'^AC_.*')

# Step 2: Separate rows into three groups
dd_rows = df[dd_mask]
other_rows = df[~(dd_mask | ac_mask)]
ac_rows = df[ac_mask]

# Step 3: Concatenate the groups to create the reordered DataFrame
reordered_df = pd.concat([dd_rows, other_rows, ac_rows])


In [87]:
def unique(gene_list):
    '''This function will remove duplicates in a list while maintaing the order of first appearance'''
    seen = set()
    return [x for x in gene_list if x not in seen and not seen.add(x)]

# Initialize all_genes with the initial list of genes
all_genes = curriculum_dict[f'Model all final genes'][0]['top 300 genes']

def gene_ranker(df, all_genes, rank_ordered_genes = [], N=None, show=False):
    '''This function will take a df assuming the column order and row order respectively indicate importance of the gene or cell in question
    the list of all_genes that should ultimately be compared against must also be specified
    rank_ordered_genes is the by default empty list of genes that are being added, however, a list of genes can be provided to fix them at the top
    N is the minimum number of genes that can be returned in the rank orderered gene list
    show will print the final list if desired'''

    # Iterate through columns until rank_ordered_genes contains all_genes
    j = 0
    while set(rank_ordered_genes) != set(all_genes):
        # Get the unique genes in the next column of the input df, that is look at the genes of next importance level
        next_column = unique(list(df.iloc[:, j]))
        # Add the newly found genes to the existing gene list
        rank_ordered_genes.extend(next_column)
        # Remove duplicates while maintaing the order
        rank_ordered_genes = unique(rank_ordered_genes)
        # Iterate to the next column of genes
        j += 1
        # Check if the minimum number of genes desired has been found
        if N != None:
            if len(rank_ordered_genes) >= N:
                break
    if show:
        # Now, rank_ordered_genes contains all the unique genes from reordered_df in the order they appear
        print(rank_ordered_genes)
    return rank_ordered_genes

rank_ordered_genes = gene_ranker(reordered_df, all_genes, show=True)
print(len(rank_ordered_genes))

['Kcnip4', 'Ppp1r17', 'Syndig1l', 'Tfap2d', 'Gabrr3', 'Neurod2', 'Scgn', 'Eomes', 'Isl2', 'Isl1', 'Zic1', 'Sox9', 'Pou3f1', 'Kcnab1', 'Zeb2', 'Pcdh17', 'Spon1', 'C1ql3', 'Gabrg3', 'Tacr3', 'Vsx1', 'Grm5', 'Meis2', 'Glra1', 'Stk32a', 'Tbx20', 'Slc17a8', 'Cpne4', 'Mmp9', 'Igfbp5', 'Rprm', 'Gabrb1', 'Slc1a3', 'Pou4f3', 'Crym', 'Grm8', 'Syt2', 'Syt6', 'Lrrtm1', 'Cd24a', 'Opn4', 'Cck', 'Lypd1', 'Hes1', 'Gsg1', 'Cdh8', 'Gjd2', 'Gm4792', 'Maf', 'Rab3b', 'Pax6', 'Nfia', 'Nos1', 'Kcnb2', 'Gprc5b', 'Kcna1', 'Igfbp4', 'Fam19a1', 'Gabra4', 'Cdkn1c', 'Penk', 'Slitrk6', 'Kcnc4', 'Tmeff2', 'Foxp2', 'Calb1', 'A730046J19Rik', 'Opn1sw', 'Cd83', 'Pcdh11x', 'Tgfb2', 'Sparc', 'Lxn', 'Onecut1', 'Pantr1', 'Tnnt1', 'Gal', 'Pvalb', 'Cd9', 'Nfix', 'Lmo2', 'Kcnd2', 'Sh3bgr', 'Aif1', 'Gjc1', 'Opn1mw', 'Etv1', 'Gabra1', 'Tpbg', 'Amigo2', 'Gabra3', 'Th', 'Ccdc88b', 'Grik1', 'Serpine2', 'Slc17a7', 'Igfbp7', 'Otor', 'Kcnh1', 'Rlbp1', 'Fam19a3', 'Necab1', 'Lamp5', 'Nell1', 'Gad2', 'Kcnip1', 'Prkca', 'Evc2', 'Gngt1', '

In [88]:
rank_ordered_genes_byCell = []
for i in range(1,reordered_df.shape[0]):
    df_subset = reordered_df.iloc[:i,:]
    rank_ordered_genes_byCell = gene_ranker(df_subset, all_genes, rank_ordered_genes_byCell, N=i*3, show=True)
    print("#"*100)
print(len(rank_ordered_genes_byCell))

# Open the CSV file in write mode
csv_file_path = '/home/sam/scRNAseq/Xenium/gene_list_order_of_importance.csv'
with open(csv_file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    
    # Write the list as a single row in the CSV file
    writer.writerow(rank_ordered_genes_byCell)

['Kcnip4', 'Isl2', 'Gabrg3']
####################################################################################################
['Kcnip4', 'Isl2', 'Gabrg3', 'Ppp1r17', 'Isl1', 'Zic1']
####################################################################################################
['Kcnip4', 'Isl2', 'Gabrg3', 'Ppp1r17', 'Isl1', 'Zic1', 'Syndig1l', 'Grm5', 'Tfap2d']
####################################################################################################
['Kcnip4', 'Isl2', 'Gabrg3', 'Ppp1r17', 'Isl1', 'Zic1', 'Syndig1l', 'Grm5', 'Tfap2d', 'Glra1', 'Stk32a', 'Slc17a8', 'Cpne4', 'Sox9']
####################################################################################################
['Kcnip4', 'Isl2', 'Gabrg3', 'Ppp1r17', 'Isl1', 'Zic1', 'Syndig1l', 'Grm5', 'Tfap2d', 'Glra1', 'Stk32a', 'Slc17a8', 'Cpne4', 'Sox9', 'Pou3f1', 'Mmp9', 'Igfbp5']
####################################################################################################
['Kcnip4', 'Isl2', 'Gabrg3', '